In [0]:
pip install yfinance

In [0]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta

In [0]:
spark.catalog.setCurrentDatabase("default")

In [0]:
cnt=spark.sql("Select max(Date) from Market").first()[0]
if cnt is None:
    start = datetime(2022, 8, 1).date()
    end=(datetime.now()+timedelta(days=1)).date()
    print("Hola")
else:
    start=spark.sql("Select max(Date) from Market").first()[0]
    end=(datetime.now()+timedelta(days=1)).date()
    print("Regular")
sn=yf.download("^BSESN",start=start,end=end)
sn.reset_index(inplace=True)
sn.columns=[col[0] if isinstance(col,tuple) else col for col in sn.columns]
sn=sn[["Date","Close"]].copy()
sn.rename(columns={"Close":"Sensex_Close"},inplace=True)
sn["Date"]=sn["Date"].dt.date
sn["Sensex_Close"]=sn["Sensex_Close"].round(2)
sn["Year"]=pd.to_datetime(sn["Date"]).dt.year
sn["Month"]=pd.to_datetime(sn["Date"]).dt.month
sn=spark.createDataFrame(sn)
sn.createOrReplaceTempView("new_data_sn")

In [0]:
%sql
--ALTER TABLE Market SET TBLPROPERTIES (
  --'delta.minReaderVersion' = '2',
  --'delta.minWriterVersion' = '5',
  --'delta.columnMapping.mode' = 'name'
--);

--alter table Market drop columns (Sensex_Daily_Returns);
--create table Market(
  --Date DATE,
  --Sensex_Close double,
  --Year int,
  --Month int
--) using delta
--partitioned by (Year,Month)

In [0]:
%sql 
merge into Market target
using new_data_sn source
on source.Date=target.Date
when matched then
update set target.Sensex_Close=source.Sensex_Close
when not matched then
insert (Date,Sensex_Close,Year,Month) values (source.Date,source.Sensex_Close,source.Year,source.Month) 

In [0]:
%sql
select Date,Sensex_Close
from Market
order by Date
limit 10;